In [1]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management
%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

%run ./my_performances.ipynb 


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data descriptor 
# reload the data fingerprint 
#=======================================================
k=10
final_clean_fingerp_PubChem_train=[]
final_clean_fingerp_PubChem_val=[]
for i in range(k):
    final_clean_fingerp_PubChem_train.append(np.load('final_clean_fingerp_PubChem_train'+ str(i)+'.npy'))
    final_clean_fingerp_PubChem_val.append(np.load('final_clean_fingerp_PubChem_val' +str(i)+'.npy'))

final_clean_fingerp_PubChem_test = np.load('final_clean_fingerp_PubChem_test.npy')

In [3]:
train_idx = np.load('train_indices.npy')
val_idx = np.load('val_indices.npy')
test_idx = np.load('test_indices.npy')

In [4]:
# load the output label 
total_train_targets =[]
total_validation_targets =[]
total_test_targets=[]
for i in range(k):
    total_train_targets.append(np.load('total_train_targets'+ str(i)+'.npy'))
    total_validation_targets.append(np.load('total_validation_targets' +str(i)+'.npy'))

total_test_targets= np.load('total_test_targets.npy')

In [5]:
# create dataloader for training (vector data)
#======================================================================================
list_data_PubChem_train =[]
list_data_PubChem_val =[]
list_data_target_train =[]
list_data_target_val =[]

for data_train, data_val, tr_targets, val_targets in zip(final_clean_fingerp_PubChem_train, final_clean_fingerp_PubChem_val,total_train_targets, total_validation_targets):
    train_loader = V_Loader(dataset = data_train, batch_size = 126)
    val_loader = V_Loader(dataset = data_val, batch_size = 126)
    
    tr_target_loader = V_Loader(dataset = tr_targets, batch_size = 126)
    val_target_loader =  V_Loader(dataset = val_targets, batch_size = 126)
    
    list_data_PubChem_train.append(train_loader)
    list_data_PubChem_val.append(val_loader)
    
    list_data_target_train.append(tr_target_loader)
    list_data_target_val.append(val_target_loader)

In [6]:
#criterion = torch.nn.CrossEntropyLoss()
#define the loss function 
criterion = torch.nn.BCELoss()

In [7]:
def train_1(train_v_loaderB,tr_target_v_loaderB, combined_model, optimizer):
    combined_model.train()

# only used for counting the lost function for data train
#====================================================================    
    
    for data_X2, data_target in zip(train_v_loaderB, tr_target_v_loaderB):  # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        #print(out)
        y_t = torch.tensor(data_target, dtype=torch.float32)
        loss = criterion(out[:,0], y_t)  # Compute the loss.
        #print(k)
        #print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss, combined_model, optimizer

# only used for counting the lost function for data validation
#====================================================================
def validation_1(val_v_loaderB,val_target_v_loaderB, combined_model):
    for data_X2, data_target in zip(val_v_loaderB, val_target_v_loaderB): # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        
        y_t = torch.tensor(data_target, dtype=torch.float32)
        val_loss = criterion(out[:,0], y_t)  # Compute the loss.
        
    return val_loss

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

# performances visualization 
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics
import math
from prettytable import PrettyTable
%run ./my_performances.ipynb 


def test_1(v_loaderB,v_target, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X2, data_target in zip (v_loaderB,v_target):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            
            list_pred.append(out_1.item())
            list_targets.append(data_target.item())
    return list_pred, list_targets

# used to count the train accuracy ,and validation accuracy when in the training mode 
def test(v_loaderB,target_v_loaderB, combined_model):
    combined_model.eval()

    correct = 0
    for data_X2, data_target in zip(v_loaderB,target_v_loaderB):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            for i,value in enumerate(out_1) :
                if value > 0.5 :
                    out_1[i] = 1
                else : out_1[i] = 0
            pred = out_1  # Use the class with highest probability.
            correct += int((pred == data_target).sum())  # Check against ground-truth labels.
    return correct / len(v_loaderB.dataset)  # Derive ratio of correct predictions.

In [9]:
def get_optimizer(gnn_model, learning_rate, optimizer_type, weight_decay=1e-4):
    if optimizer_type==1:
        optimizer = torch.optim.SGD(gnn_model.parameters(), lr=learning_rate, momentum=0.9)
    if optimizer_type==2:
        optimizer = torch.optim.Adam(gnn_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    if optimizer_type ==3 :
        optimizer = torch.optim.Adamax(gnn_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
        
    return optimizer

In [10]:
class modelB(torch.nn.Module):
    def __init__(self, input_features, output_features,dropout_rateB1,dropout_rateB2,dropout_rateB3,  
                 dense_layer1,dense_layer2, dense_layer3):
        super(modelB, self).__init__()
        self.lin1 = nn.Linear(input_features,dense_layer1)
      
        self.lin2 = nn.Linear(int(dense_layer1), dense_layer2)
        self.lin3 = nn.Linear(int(dense_layer2), dense_layer3)
        self.lin4 = nn.Linear(int(dense_layer3), output_features)
        
        self.bn1 = nn.BatchNorm1d(int(dense_layer1))
        self.bn2 = nn.BatchNorm1d(int(dense_layer2))
        self.bn3 = nn.BatchNorm1d(int(dense_layer3))
        self.dropoutB1 = dropout_rateB1
        self.dropoutB2 = dropout_rateB2
        self.dropoutB3 = dropout_rateB3
        
    def forward(self, x):
        x = self.lin1(x)
        
        x = self.bn1(x)
       
        x = F.dropout(x, p= self.dropoutB1, training=self.training)
        x = x.relu()
  #      
        x = self.lin2(x)
        x = self.bn2(x)   
        x = F.dropout(x, p= self.dropoutB2, training=self.training)
        x = x.relu()
  #      
        x = self.lin3(x)
        x = self.bn3(x)   
        x = F.dropout(x, p= self.dropoutB3, training=self.training)

        x = x.relu()
        x = self.lin4(x)
        return torch.sigmoid(x)    

In [11]:
k=10 # number of fold 
input_featuresB    = 881 # length of feature data vector 
output_featuresB   = 1
optimizer_type = 2
dropout_rateB1 = 0.15552924759769313
dropout_rateB2 =0.207127885808487
dropout_rateB3 =0.2240506664932075

dense_layer1=60
dense_layer2=30
dense_layer3=10
learning_rate = 0.015061165279742287
weight_decay=0.00017722962529205202

list_data_vec_trainB = list_data_PubChem_train
list_data_vec_valB = list_data_PubChem_val

In [12]:
# APPLY THE HYPERPARAMETER 
# RUNNING EVERY FOLD 
#=============================================================

total_history_train_loss=[]
total_history_val_loss=[]
total_history_train_acc=[]
total_history_val_acc=[]
list_trained_model =[]

for i in range(k):
        
        combined_model   = modelB(input_featuresB, output_featuresB,
                                   dropout_rateB1,dropout_rateB1,dropout_rateB3, 
                                   dense_layer1,dense_layer2, dense_layer3)
        optimizer = get_optimizer(combined_model, learning_rate, optimizer_type,weight_decay)
   
        history_train_loss=[]
        history_val_loss=[]
        history_train_acc=[]
        history_val_acc=[]
        
        for epoch in range(1,40):
            
                
            train_loss,combined_model, optimizer = train_1(list_data_vec_trainB[i],list_data_target_train[i],
                                                           combined_model,optimizer)
            val_loss                             = validation_1(list_data_vec_valB[i],list_data_target_val[i],
                                                                combined_model) 
        
            train_acc                            = test(list_data_vec_trainB[i],list_data_target_train[i],combined_model)
            val_acc                              = test(list_data_vec_valB[i],list_data_target_val[i], combined_model)
        
           
            
            history_train_loss.append(train_loss.item())
            history_val_loss.append(val_loss.item())
            history_train_acc.append(train_acc)
            history_val_acc.append(val_acc)
        
            print(f'Fold: {i}')
            print(f'Epoch: {epoch:03d}, Train Loss:, {train_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
            print(f'Epoch: {epoch:03d}, val Loss:, {val_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, val Acc: {val_acc:.4f}')
            print(" ")
        total_history_train_loss.append(history_train_loss)
        total_history_val_loss.append(history_val_loss)
        total_history_train_acc.append(history_train_acc)
        total_history_val_acc.append(history_val_acc)

        list_trained_model.append(combined_model)


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct f

Fold: 0
Epoch: 001, Train Loss:, 0.5830
Epoch: 001, Train Acc: 0.6811
Epoch: 001, val Loss:, 0.6276
Epoch: 001, val Acc: 0.6746
 
Fold: 0
Epoch: 002, Train Loss:, 0.5076
Epoch: 002, Train Acc: 0.7094
Epoch: 002, val Loss:, 0.5873
Epoch: 002, val Acc: 0.7143
 
Fold: 0
Epoch: 003, Train Loss:, 0.4672
Epoch: 003, Train Acc: 0.7270
Epoch: 003, val Loss:, 0.5838
Epoch: 003, val Acc: 0.7143
 
Fold: 0
Epoch: 004, Train Loss:, 0.4895
Epoch: 004, Train Acc: 0.7562
Epoch: 004, val Loss:, 0.5641
Epoch: 004, val Acc: 0.7857
 
Fold: 0
Epoch: 005, Train Loss:, 0.4511
Epoch: 005, Train Acc: 0.7942
Epoch: 005, val Loss:, 0.6266
Epoch: 005, val Acc: 0.7381
 
Fold: 0
Epoch: 006, Train Loss:, 0.4529
Epoch: 006, Train Acc: 0.8004
Epoch: 006, val Loss:, 0.6169
Epoch: 006, val Acc: 0.7222
 
Fold: 0
Epoch: 007, Train Loss:, 0.4237
Epoch: 007, Train Acc: 0.8127
Epoch: 007, val Loss:, 0.6333
Epoch: 007, val Acc: 0.6746
 
Fold: 0
Epoch: 008, Train Loss:, 0.4159
Epoch: 008, Train Acc: 0.8216
Epoch: 008, val Loss

Fold: 1
Epoch: 028, Train Loss:, 0.1779
Epoch: 028, Train Acc: 0.8719
Epoch: 028, val Loss:, 1.0927
Epoch: 028, val Acc: 0.5873
 
Fold: 1
Epoch: 029, Train Loss:, 0.2726
Epoch: 029, Train Acc: 0.8913
Epoch: 029, val Loss:, 1.1982
Epoch: 029, val Acc: 0.6984
 
Fold: 1
Epoch: 030, Train Loss:, 0.1784
Epoch: 030, Train Acc: 0.9293
Epoch: 030, val Loss:, 1.2382
Epoch: 030, val Acc: 0.6349
 
Fold: 1
Epoch: 031, Train Loss:, 0.2034
Epoch: 031, Train Acc: 0.9143
Epoch: 031, val Loss:, 1.1484
Epoch: 031, val Acc: 0.6032
 
Fold: 1
Epoch: 032, Train Loss:, 0.2293
Epoch: 032, Train Acc: 0.9408
Epoch: 032, val Loss:, 1.2330
Epoch: 032, val Acc: 0.6429
 
Fold: 1
Epoch: 033, Train Loss:, 0.1842
Epoch: 033, Train Acc: 0.9284
Epoch: 033, val Loss:, 1.2966
Epoch: 033, val Acc: 0.6508
 
Fold: 1
Epoch: 034, Train Loss:, 0.2073
Epoch: 034, Train Acc: 0.9284
Epoch: 034, val Loss:, 1.4135
Epoch: 034, val Acc: 0.6429
 
Fold: 1
Epoch: 035, Train Loss:, 0.1643
Epoch: 035, Train Acc: 0.9461
Epoch: 035, val Loss

Fold: 3
Epoch: 014, Train Loss:, 0.3563
Epoch: 014, Train Acc: 0.8569
Epoch: 014, val Loss:, 0.8873
Epoch: 014, val Acc: 0.5635
 
Fold: 3
Epoch: 015, Train Loss:, 0.3362
Epoch: 015, Train Acc: 0.8834
Epoch: 015, val Loss:, 0.7803
Epoch: 015, val Acc: 0.6429
 
Fold: 3
Epoch: 016, Train Loss:, 0.2210
Epoch: 016, Train Acc: 0.9055
Epoch: 016, val Loss:, 0.9577
Epoch: 016, val Acc: 0.6587
 
Fold: 3
Epoch: 017, Train Loss:, 0.2772
Epoch: 017, Train Acc: 0.8631
Epoch: 017, val Loss:, 1.0913
Epoch: 017, val Acc: 0.6349
 
Fold: 3
Epoch: 018, Train Loss:, 0.2753
Epoch: 018, Train Acc: 0.8931
Epoch: 018, val Loss:, 1.1446
Epoch: 018, val Acc: 0.6429
 
Fold: 3
Epoch: 019, Train Loss:, 0.3181
Epoch: 019, Train Acc: 0.8728
Epoch: 019, val Loss:, 1.3003
Epoch: 019, val Acc: 0.6508
 
Fold: 3
Epoch: 020, Train Loss:, 0.3160
Epoch: 020, Train Acc: 0.8127
Epoch: 020, val Loss:, 1.2080
Epoch: 020, val Acc: 0.6032
 
Fold: 3
Epoch: 021, Train Loss:, 0.2463
Epoch: 021, Train Acc: 0.8993
Epoch: 021, val Loss

Fold: 5
Epoch: 001, Train Loss:, 0.5646
Epoch: 001, Train Acc: 0.6890
Epoch: 001, val Loss:, 0.6282
Epoch: 001, val Acc: 0.6587
 
Fold: 5
Epoch: 002, Train Loss:, 0.5298
Epoch: 002, Train Acc: 0.7147
Epoch: 002, val Loss:, 0.6285
Epoch: 002, val Acc: 0.6905
 
Fold: 5
Epoch: 003, Train Loss:, 0.4895
Epoch: 003, Train Acc: 0.7456
Epoch: 003, val Loss:, 0.6157
Epoch: 003, val Acc: 0.7063
 
Fold: 5
Epoch: 004, Train Loss:, 0.4811
Epoch: 004, Train Acc: 0.7650
Epoch: 004, val Loss:, 0.6436
Epoch: 004, val Acc: 0.6349
 
Fold: 5
Epoch: 005, Train Loss:, 0.4092
Epoch: 005, Train Acc: 0.7712
Epoch: 005, val Loss:, 0.6505
Epoch: 005, val Acc: 0.6905
 
Fold: 5
Epoch: 006, Train Loss:, 0.4538
Epoch: 006, Train Acc: 0.7783
Epoch: 006, val Loss:, 0.7354
Epoch: 006, val Acc: 0.6587
 
Fold: 5
Epoch: 007, Train Loss:, 0.3839
Epoch: 007, Train Acc: 0.8180
Epoch: 007, val Loss:, 0.7438
Epoch: 007, val Acc: 0.6667
 
Fold: 5
Epoch: 008, Train Loss:, 0.3649
Epoch: 008, Train Acc: 0.8074
Epoch: 008, val Loss

Fold: 6
Epoch: 029, Train Loss:, 0.1955
Epoch: 029, Train Acc: 0.9170
Epoch: 029, val Loss:, 1.3428
Epoch: 029, val Acc: 0.6429
 
Fold: 6
Epoch: 030, Train Loss:, 0.2031
Epoch: 030, Train Acc: 0.9355
Epoch: 030, val Loss:, 1.2206
Epoch: 030, val Acc: 0.6349
 
Fold: 6
Epoch: 031, Train Loss:, 0.2101
Epoch: 031, Train Acc: 0.9196
Epoch: 031, val Loss:, 1.3082
Epoch: 031, val Acc: 0.6667
 
Fold: 6
Epoch: 032, Train Loss:, 0.1180
Epoch: 032, Train Acc: 0.9461
Epoch: 032, val Loss:, 1.3696
Epoch: 032, val Acc: 0.6508
 
Fold: 6
Epoch: 033, Train Loss:, 0.2753
Epoch: 033, Train Acc: 0.9399
Epoch: 033, val Loss:, 1.5350
Epoch: 033, val Acc: 0.6032
 
Fold: 6
Epoch: 034, Train Loss:, 0.1914
Epoch: 034, Train Acc: 0.9435
Epoch: 034, val Loss:, 1.3802
Epoch: 034, val Acc: 0.5556
 
Fold: 6
Epoch: 035, Train Loss:, 0.1776
Epoch: 035, Train Acc: 0.9364
Epoch: 035, val Loss:, 1.4939
Epoch: 035, val Acc: 0.6508
 
Fold: 6
Epoch: 036, Train Loss:, 0.1686
Epoch: 036, Train Acc: 0.9284
Epoch: 036, val Loss

Fold: 8
Epoch: 016, Train Loss:, 0.2955
Epoch: 016, Train Acc: 0.8808
Epoch: 016, val Loss:, 0.9888
Epoch: 016, val Acc: 0.6160
 
Fold: 8
Epoch: 017, Train Loss:, 0.2971
Epoch: 017, Train Acc: 0.8667
Epoch: 017, val Loss:, 1.0854
Epoch: 017, val Acc: 0.5760
 
Fold: 8
Epoch: 018, Train Loss:, 0.2153
Epoch: 018, Train Acc: 0.9109
Epoch: 018, val Loss:, 1.1590
Epoch: 018, val Acc: 0.6080
 
Fold: 8
Epoch: 019, Train Loss:, 0.1911
Epoch: 019, Train Acc: 0.9303
Epoch: 019, val Loss:, 1.3010
Epoch: 019, val Acc: 0.6320
 
Fold: 8
Epoch: 020, Train Loss:, 0.2538
Epoch: 020, Train Acc: 0.8994
Epoch: 020, val Loss:, 1.1950
Epoch: 020, val Acc: 0.6480
 
Fold: 8
Epoch: 021, Train Loss:, 0.2430
Epoch: 021, Train Acc: 0.8658
Epoch: 021, val Loss:, 1.1670
Epoch: 021, val Acc: 0.6560
 
Fold: 8
Epoch: 022, Train Loss:, 0.2678
Epoch: 022, Train Acc: 0.9109
Epoch: 022, val Loss:, 1.1532
Epoch: 022, val Acc: 0.6640
 
Fold: 8
Epoch: 023, Train Loss:, 0.2146
Epoch: 023, Train Acc: 0.9117
Epoch: 023, val Loss

In [14]:
# RUNNING THE 10 PRETRAINED MODEL USING INDEPENDENT DATA
#======================================================================
nCV= 10 # ten crossfold validation 
list_fold_pred =[]
list_fold_targets =[]


v_test_loaderB = V_Loader(dataset = final_clean_fingerp_PubChem_test[0], batch_size = 1)
v_test_target = V_Loader(dataset = total_test_targets, batch_size = 1)

for combined_model in list_trained_model:  
    list_pred, list_targets = test_1(v_test_loaderB,v_test_target,combined_model)
    list_fold_pred.append(list_pred)
    list_fold_targets.append(list_targets)
    


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [15]:
# GET THE PERFORMANCES FROM THE TEST
#========================================================================
total_performances = performances(list_fold_pred, list_fold_targets, nCV)
list_bal_acc = []
for sen, spec in zip (total_performances[1] , total_performances[2]):
    bal_acc = (sen + spec)/2
    list_bal_acc.append(bal_acc)
                
statistics.mean(list_bal_acc)

0.652245

In [16]:
import statistics
from prettytable import PrettyTable
perf = total_performances
model_title = 'Test Perf FCNN'
data_type ='PubChem'
Create_Tables(perf, model_title, data_type)

+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | m_ACC |  m_SN |  m_SP | m_MCC | m_AUC | m_Kappa | m_AUROC | m_Bal_ACC |  m_F1 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |  PubChem  | 0.694 | 0.763 | 0.542 |  0.3  |  0.71 |  0.299  |   0.71  |   0.652   | 0.774 |
+----------------+-----------+-------+-------+-------+-------+-------+---------+---------+-----------+-------+
+----------------+-----------+-------+-------+------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | e_ACC |  e_SN |  SP  | e_MCC | e_AUC | e_Kappa | e_AUROC | e_Bal_ACC |  e_F1 |
+----------------+-----------+-------+-------+------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |  PubChem  | 0.021 | 0.039 | 0.04 | 0.033 | 0.013 |  0.034  |  0.013  |   0.016   | 0.774 |
+----